# Stance Detection for the Fake News Challenge

## Identifying Textual Relationships with Deep Neural Nets

### Check the problem context [here](https://drive.google.com/open?id=1KfWaZyQdGBw8AUTacJ2yY86Yxgw2Xwq0).

### Download files required for the project from [here](https://drive.google.com/open?id=10yf39ifEwVihw4xeJJR60oeFBY30Y5J8).

## Step1: Load the given dataset  

1. Mount the google drive

2. Import Glove embeddings

3. Import the test and train datasets

### Mount the google drive to access required project files

Run the below commands

In [0]:
from google.colab import drive

In [2]:
drive.mount('/content/drive/')

Go to this URL in a browser: https://accounts.google.com/o/oauth2/auth?client_id=947318989803-6bn6qk8qdgf4n4g3pfee6491hc0brc4i.apps.googleusercontent.com&redirect_uri=urn%3Aietf%3Awg%3Aoauth%3A2.0%3Aoob&scope=email%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fdocs.test%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fdrive%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fdrive.photos.readonly%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fpeopleapi.readonly&response_type=code

Enter your authorization code:
··········
Mounted at /content/drive/


#### Path for Project files on google drive

**Note:** You need to change this path according where you have kept the files in google drive. 

In [0]:
project_path = "/content/drive/My Drive/2 - R10 Project Sequential Model Fake News - Rohit S/"

In [0]:
import os
os.chdir(project_path)

### Loading the Glove Embeddings

In [0]:
from zipfile import ZipFile
with ZipFile(project_path+'glove.6B.zip', 'r') as z:
  z.extractall()

# Load the dataset [5 Marks]

1. Using [read_csv()](https://pandas.pydata.org/pandas-docs/stable/generated/pandas.read_csv.html) in pandas load the given train datasets files **`train_bodies.csv`** and **`train_stances.csv`**

2. Using [merge](https://pandas.pydata.org/pandas-docs/stable/generated/pandas.DataFrame.merge.html) command in pandas merge the two datasets based on the Body ID. 

Note: Save the final merged dataset in a dataframe with name **`dataset`**.

In [0]:
# All definitions and imports
import pandas as pd

In [7]:
train_bodies = pd.read_csv('train_bodies.csv')
train_bodies.head(2)

,Body ID,articleBody
0,0,A small meteorite crashed into a wooded area i...
1,4,Last week we hinted at what was to come as Ebo...


In [8]:
train_stances = pd.read_csv('train_stances.csv')
train_stances.head(2)

,Headline,Body ID,Stance
0,Police find mass graves with at least '15 bodi...,712,unrelated
1,Hundreds of Palestinians flee floods in Gaza a...,158,agree


In [0]:
dataset = train_bodies.merge(train_stances, left_on='Body ID', right_on='Body ID')


<h2> Check1:</h2>
  
<h3> You should see the below output if you run `dataset.head()` command as given below </h3>

In [10]:
dataset.head()

,Body ID,articleBody,Headline,Stance
0,0,A small meteorite crashed into a wooded area i...,"Soldier shot, Parliament locked down after gun...",unrelated
1,0,A small meteorite crashed into a wooded area i...,Tourist dubbed ‘Spider Man’ after spider burro...,unrelated
2,0,A small meteorite crashed into a wooded area i...,Luke Somers 'killed in failed rescue attempt i...,unrelated
3,0,A small meteorite crashed into a wooded area i...,BREAKING: Soldier shot at War Memorial in Ottawa,unrelated
4,0,A small meteorite crashed into a wooded area i...,Giant 8ft 9in catfish weighing 19 stone caught...,unrelated


## Step2: Data Pre-processing and setting some hyper parameters needed for model


#### Run the code given below to set the required parameters.

1. `MAX_SENTS` = Maximum no.of sentences to consider in an article.

2. `MAX_SENT_LENGTH` = Maximum no.of words to consider in a sentence.

3. `MAX_NB_WORDS` = Maximum no.of words in the total vocabualry.

4. `MAX_SENTS_HEADING` = Maximum no.of sentences to consider in a heading of an article.

In [0]:
MAX_NB_WORDS = 20000
MAX_SENTS = 20
MAX_SENTS_HEADING = 1
MAX_SENT_LENGTH = 20
VALIDATION_SPLIT = 0.2

### Download the `Punkt` from nltk using the commands given below. This is for sentence tokenization.

For more info on how to use it, read [this](https://stackoverflow.com/questions/35275001/use-of-punktsentencetokenizer-in-nltk).



In [12]:
import nltk
nltk.download('punkt')

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.


True

# Tokenizing the text and loading the pre-trained Glove word embeddings for each token  [5 marks] 

Keras provides [Tokenizer API](https://keras.io/preprocessing/text/) for preparing text. Read it before going any further.

#### Import the Tokenizer from keras preprocessing text

In [13]:
import glob

from sklearn.utils import shuffle
import numpy as np

from keras.preprocessing.text import Tokenizer
from keras.models import Sequential, load_model
from keras.layers import LSTM, Dense, Dropout, Embedding, Masking, Bidirectional
from keras.preprocessing.text import text_to_word_sequence
from keras.optimizers import Adam
from keras import backend
import re

Using TensorFlow backend.


#### Initialize the Tokenizer class with maximum vocabulary count as `MAX_NB_WORDS` initialized at the start of step2. 

In [0]:
tokenizer = Tokenizer(num_words=MAX_NB_WORDS, lower=True, filters='!"#$%&()*+,-./:;<=>?@[\\]^_`{|}~\t\n')

#### Now, using fit_on_texts() from Tokenizer class, lets encode the data 

Note: We need to fit articleBody and Headline also to cover all the words.

In [15]:
# Create a list of unique articleBody and Headline texts

codetexts = []
for ab in dataset.articleBody.unique():
  codetexts.append(ab)

for hl in dataset.Headline.unique():
  codetexts.append(hl)

print(len(codetexts))

3317


In [0]:
tokenizer.fit_on_texts(codetexts)

#### fit_on_texts() gives the following attributes in the output as given [here](https://faroit.github.io/keras-docs/1.2.2/preprocessing/text/).

* **word_counts:** dictionary mapping words (str) to the number of times they appeared on during fit. Only set after fit_on_texts was called.

* **word_docs:** dictionary mapping words (str) to the number of documents/texts they appeared on during fit. Only set after fit_on_texts was called.

* **word_index:** dictionary mapping words (str) to their rank/index (int). Only set after fit_on_texts was called.

* **document_count:** int. Number of documents (texts/sequences) the tokenizer was trained on. Only set after fit_on_texts or fit_on_sequences was called.



### Now, tokenize the sentences using nltk sent_tokenize() and encode the senteces with the ids we got form the above `t.word_index`

Initialise 2 lists with names `texts` and `articles`.

```
texts = [] to store text of article as it is.

articles = [] split the above text into a list of sentences.
```

In [17]:
texts = []
for ab in dataset.articleBody:
  texts.append(ab)
print(len(texts))

49972


In [18]:
articles = []
for tx in texts:
  articles.append(nltk.sent_tokenize(tx))
print(len(articles))

49972


In [19]:
texts[0]

'A small meteorite crashed into a wooded area in Nicaragua\'s capital of Managua overnight, the government said Sunday. Residents reported hearing a mysterious boom that left a 16-foot deep crater near the city\'s airport, the Associated Press reports. \n\nGovernment spokeswoman Rosario Murillo said a committee formed by the government to study the event determined it was a "relatively small" meteorite that "appears to have come off an asteroid that was passing close to Earth." House-sized asteroid 2014 RC, which measured 60 feet in diameter, skimmed the Earth this weekend, ABC News reports. \nMurillo said Nicaragua will ask international experts to help local scientists in understanding what happened.\n\nThe crater left by the meteorite had a radius of 39 feet and a depth of 16 feet,  said Humberto Saballos, a volcanologist with the Nicaraguan Institute of Territorial Studies who was on the committee. He said it is still not clear if the meteorite disintegrated or was buried.\n\nHumbe

In [20]:
articles[0]

["A small meteorite crashed into a wooded area in Nicaragua's capital of Managua overnight, the government said Sunday.",
 "Residents reported hearing a mysterious boom that left a 16-foot deep crater near the city's airport, the Associated Press reports.",
 'Government spokeswoman Rosario Murillo said a committee formed by the government to study the event determined it was a "relatively small" meteorite that "appears to have come off an asteroid that was passing close to Earth."',
 'House-sized asteroid 2014 RC, which measured 60 feet in diameter, skimmed the Earth this weekend, ABC News reports.',
 'Murillo said Nicaragua will ask international experts to help local scientists in understanding what happened.',
 'The crater left by the meteorite had a radius of 39 feet and a depth of 16 feet,  said Humberto Saballos, a volcanologist with the Nicaraguan Institute of Territorial Studies who was on the committee.',
 'He said it is still not clear if the meteorite disintegrated or was bu

## Check 2:

first element of texts and articles should be as given below. 

In [21]:
texts[0]

'A small meteorite crashed into a wooded area in Nicaragua\'s capital of Managua overnight, the government said Sunday. Residents reported hearing a mysterious boom that left a 16-foot deep crater near the city\'s airport, the Associated Press reports. \n\nGovernment spokeswoman Rosario Murillo said a committee formed by the government to study the event determined it was a "relatively small" meteorite that "appears to have come off an asteroid that was passing close to Earth." House-sized asteroid 2014 RC, which measured 60 feet in diameter, skimmed the Earth this weekend, ABC News reports. \nMurillo said Nicaragua will ask international experts to help local scientists in understanding what happened.\n\nThe crater left by the meteorite had a radius of 39 feet and a depth of 16 feet,  said Humberto Saballos, a volcanologist with the Nicaraguan Institute of Territorial Studies who was on the committee. He said it is still not clear if the meteorite disintegrated or was buried.\n\nHumbe

In [22]:
articles[0]

["A small meteorite crashed into a wooded area in Nicaragua's capital of Managua overnight, the government said Sunday.",
 "Residents reported hearing a mysterious boom that left a 16-foot deep crater near the city's airport, the Associated Press reports.",
 'Government spokeswoman Rosario Murillo said a committee formed by the government to study the event determined it was a "relatively small" meteorite that "appears to have come off an asteroid that was passing close to Earth."',
 'House-sized asteroid 2014 RC, which measured 60 feet in diameter, skimmed the Earth this weekend, ABC News reports.',
 'Murillo said Nicaragua will ask international experts to help local scientists in understanding what happened.',
 'The crater left by the meteorite had a radius of 39 feet and a depth of 16 feet,  said Humberto Saballos, a volcanologist with the Nicaraguan Institute of Territorial Studies who was on the committee.',
 'He said it is still not clear if the meteorite disintegrated or was bu

# Now iterate through each article and each sentence to encode the words into ids using t.word_index  [5 marks] 

Here, to get words from sentence you can use `text_to_word_sequence` from keras preprocessing text.

1. Import text_to_word_sequence

2. Initialize a variable of shape (no.of articles, MAX_SENTS, MAX_SENT_LENGTH) with name `data` with zeros first (you can use numpy [np.zeros](https://docs.scipy.org/doc/numpy/reference/generated/numpy.zeros.html) to initialize with all zeros)and then update it while iterating through the words and sentences in each article.

In [0]:
# summarize what was learned

word_counts = tokenizer.word_counts
document_counts = tokenizer.document_count
word_idx = tokenizer.word_index
word_docs = tokenizer.word_docs

idx_word = tokenizer.index_word

In [0]:
lenart = len(articles)
data = np.zeros((lenart,MAX_SENTS,MAX_SENT_LENGTH))

In [0]:
art_idx = 0
for art in articles:  
  #if(len(art)>1): print(art_idx, len(art), art)
  sent_idx = 0
  for sent in art:
    #print(len(sent), sent)    
    word_idx = 0
    for word in text_to_word_sequence(sent):
      #print(word, t.word_index.get(word))
      data[art_idx][sent_idx][word_idx] = tokenizer.word_index.get(word)
      word_idx += 1
      if(word_idx == MAX_SENT_LENGTH): break
    sent_idx += 1
    if(sent_idx == MAX_SENTS): break
  art_idx += 1
        

### Check 3:

Accessing first element in data should give something like given below.

In [27]:
data[0, :, :]

array([[3.0000e+00, 4.0400e+02, 4.5300e+02, 6.3210e+03, 8.2000e+01,
        3.0000e+00, 3.6170e+03, 3.1300e+02, 5.0000e+00, 3.9070e+03,
        3.7600e+02, 4.0000e+00, 1.5060e+03, 2.9310e+03, 1.0000e+00,
        1.0300e+02, 1.5000e+01, 4.2000e+02, 0.0000e+00, 0.0000e+00],
       [8.2600e+02, 9.4000e+01, 1.3300e+03, 3.0000e+00, 2.3870e+03,
        2.3880e+03, 7.0000e+00, 1.7400e+02, 3.0000e+00, 9.3100e+02,
        1.4230e+03, 1.8800e+03, 7.6500e+02, 1.9100e+02, 1.0000e+00,
        3.4830e+03, 4.1400e+02, 1.0000e+00, 6.6800e+02, 2.5300e+02],
       [1.0300e+02, 1.0840e+03, 4.0630e+03, 2.6190e+03, 1.5000e+01,
        3.0000e+00, 1.2240e+03, 3.2450e+03, 1.9000e+01, 1.0000e+00,
        1.0300e+02, 2.0000e+00, 1.9180e+03, 1.0000e+00, 5.3100e+02,
        2.6200e+03, 1.4000e+01, 1.0000e+01, 3.0000e+00, 3.1270e+03],
       [1.8900e+02, 3.3720e+03, 8.4200e+02, 1.6200e+02, 2.3360e+03,
        4.3000e+01, 5.5330e+03, 1.8370e+03, 1.0370e+03, 5.0000e+00,
        1.0570e+04, 1.6599e+04, 1.0000e+00, 6

# Repeat the same process for the `Headings` as well. Use variables with names `texts_heading` and `articles_heading` accordingly. [5 marks] 

In [28]:
texts_heading = []
for Hl in dataset.Headline:
  texts_heading.append(Hl)
print(len(texts_heading))

49972


In [29]:
articles_heading = []
for th in texts_heading:
  articles_heading.append(nltk.sent_tokenize(th))
print(len(articles_heading))

49972


In [30]:
texts_heading[0]

'Soldier shot, Parliament locked down after gunfire erupts at war memorial'

In [32]:
articles_heading[0]

['Soldier shot, Parliament locked down after gunfire erupts at war memorial']

In [0]:
data_heading = np.zeros((len(articles_heading), MAX_SENTS, MAX_SENT_LENGTH), dtype=int)

In [0]:
art_idx = 0
for art in articles_heading:  
  #if(len(art)>1): print(art_idx, len(art), art)
  sent_idx = 0
  for sent in art:
    #print(len(sent), sent)    
    word_idx = 0
    for word in text_to_word_sequence(sent):
      #print(word, t.word_index.get(word))
      data_heading[art_idx][sent_idx][word_idx] = tokenizer.word_index.get(word)
      word_idx += 1
      if(word_idx == MAX_SENT_LENGTH): break
    sent_idx += 1
    if(sent_idx == MAX_SENTS): break
  art_idx += 1

In [37]:
data_heading[0, :, :]

array([[  593,   233,   279,  4026,   169,    34,  1359, 12475,    21,
          230,   714,     0,     0,     0,     0,     0,     0,     0,
            0,     0],
       [    0,     0,     0,     0,     0,     0,     0,     0,     0,
            0,     0,     0,     0,     0,     0,     0,     0,     0,
            0,     0],
       [    0,     0,     0,     0,     0,     0,     0,     0,     0,
            0,     0,     0,     0,     0,     0,     0,     0,     0,
            0,     0],
       [    0,     0,     0,     0,     0,     0,     0,     0,     0,
            0,     0,     0,     0,     0,     0,     0,     0,     0,
            0,     0],
       [    0,     0,     0,     0,     0,     0,     0,     0,     0,
            0,     0,     0,     0,     0,     0,     0,     0,     0,
            0,     0],
       [    0,     0,     0,     0,     0,     0,     0,     0,     0,
            0,     0,     0,     0,     0,     0,     0,     0,     0,
            0,     0],
       [  

### Now the features are ready, lets make the labels ready for the model to process.

### Convert labels into one-hot vectors

You can use [get_dummies](https://pandas.pydata.org/pandas-docs/stable/generated/pandas.get_dummies.html) in pandas to create one-hot vectors.

In [38]:
one_hot = pd.get_dummies(dataset.Stance)
one_hot.head()

,agree,disagree,discuss,unrelated
0,0,0,0,1
1,0,0,0,1
2,0,0,0,1
3,0,0,0,1
4,0,0,0,1


In [39]:
labels = np.array(one_hot)
print(labels)

[[0 0 0 1]
 [0 0 0 1]
 [0 0 0 1]
 ...
 [1 0 0 0]
 [1 0 0 0]
 [1 0 0 0]]


### Check 4:

The shape of data and labels shoould match the given below numbers.

In [40]:
print('Shape of data tensor:', data.shape)
print('Shape of label tensor:', labels.shape)

Shape of data tensor: (49972, 20, 20)
Shape of label tensor: (49972, 4)


### Shuffle the data

In [0]:
## get numbers upto no.of articles
indices = np.arange(data.shape[0])
## shuffle the numbers
np.random.shuffle(indices)

In [0]:
## shuffle the data
data = data[indices]
data_heading = data_heading[indices]
## shuffle the labels according to data
labels = labels[indices]

### Split into train and validation sets. Split the train set 80:20 ratio to get the train and validation sets.


Use the variable names as given below:

x_train, x_val - for body of articles.

x-heading_train, x_heading_val - for heading of articles.

y_train - for training labels.

y_val - for validation labels.



In [43]:
train_idx = round(data.shape[0]*0.8)
train_idx

39978

In [44]:
x_train = data[:train_idx]
x_val = data[train_idx:]
len(x_train)+len(x_val)

49972

In [45]:
x_heading_train = data_heading[:train_idx]
x_heading_val = data_heading[train_idx:]
len(x_heading_train)+len(x_heading_val)

49972

In [46]:
y_train = labels[:train_idx]
y_val = labels[train_idx:]
len(y_train)+len(y_val)

49972

### Check 5:

The shape of x_train, x_val, y_train and y_val should match the below numbers.

In [47]:
print(x_train.shape)
print(y_train.shape)

print(x_val.shape)
print(y_val.shape)

(39978, 20, 20)
(39978, 4)
(9994, 20, 20)
(9994, 4)


### Create embedding matrix with the glove embeddings


Run the below code to create embedding_matrix which has all the words and their glove embedding if present in glove word list.

In [49]:
# load the whole embedding into memory
embeddings_index = dict()
f = open('./glove.6B.100d.txt')
for line in f:
	values = line.split()
	word = values[0]
	coefs = np.asarray(values[1:], dtype='float32')
	embeddings_index[word] = coefs
f.close()
print('Loaded %s word vectors.' % len(embeddings_index))

# create a weight matrix for words in training docs
vocab_size = len(tokenizer.word_index)
embedding_matrix = np.zeros((vocab_size, 100))


for word, i in tokenizer.word_index.items():
	embedding_vector = embeddings_index.get(word)
	if embedding_vector is not None:
		embedding_matrix[i] = embedding_vector

Loaded 400000 word vectors.


# Try the sequential model approach and report the accuracy score. [10 marks]  

### Import layers from Keras to build the model

In [0]:
from keras.models import Model
from keras.layers import Input, LSTM, Dense

### Model

In [51]:
# Define an input sequence and process it.
data_inputs = Input(shape=(None, 20))
encoder_lstm = LSTM(256, return_state=True)
encoder_outputs, encoder_state_h, encoder_state_c = encoder_lstm(data_inputs)
# We discard `encoder_outputs` and only keep the states.
encoder_states = [encoder_state_h, encoder_state_c]
encoder_model = Model(data_inputs, encoder_states)
encoder_model.summary()

W0803 09:47:51.424183 139741922998144 deprecation_wrapper.py:119] From /usr/local/lib/python3.6/dist-packages/keras/backend/tensorflow_backend.py:74: The name tf.get_default_graph is deprecated. Please use tf.compat.v1.get_default_graph instead.

W0803 09:47:51.453271 139741922998144 deprecation_wrapper.py:119] From /usr/local/lib/python3.6/dist-packages/keras/backend/tensorflow_backend.py:517: The name tf.placeholder is deprecated. Please use tf.compat.v1.placeholder instead.

W0803 09:47:51.459006 139741922998144 deprecation_wrapper.py:119] From /usr/local/lib/python3.6/dist-packages/keras/backend/tensorflow_backend.py:4138: The name tf.random_uniform is deprecated. Please use tf.random.uniform instead.



_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_1 (InputLayer)         (None, None, 20)          0         
_________________________________________________________________
lstm_1 (LSTM)                [(None, 256), (None, 256) 283648    
Total params: 283,648
Trainable params: 283,648
Non-trainable params: 0
_________________________________________________________________


In [52]:
# Set up the decoder, using `encoder_states` as initial state.
data_heading_inputs = Input(shape=(None, 20))
# We set up our decoder to not return full output sequences,
# and to return internal states. We don't use the
# return states in the training model.
decoder_lstm = LSTM(256, return_sequences=False, return_state=True)
decoder_outputs, _, _ = decoder_lstm(data_heading_inputs, initial_state=encoder_states)
decoder_dense = Dense(4, activation='softmax')
decoder_outputs = decoder_dense(decoder_outputs)
model = Model([data_inputs, data_heading_inputs], decoder_outputs)
model.summary()

__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_1 (InputLayer)            (None, None, 20)     0                                            
__________________________________________________________________________________________________
input_2 (InputLayer)            (None, None, 20)     0                                            
__________________________________________________________________________________________________
lstm_1 (LSTM)                   [(None, 256), (None, 283648      input_1[0][0]                    
__________________________________________________________________________________________________
lstm_2 (LSTM)                   [(None, 256), (None, 283648      input_2[0][0]                    
                                                                 lstm_1[0][1]                     
          

### Compile and fit the model

In [53]:
# Run training
%%time
model.compile(optimizer='rmsprop', loss='categorical_crossentropy')
model.fit([x_train, x_heading_train], y_train, batch_size=64, epochs=25)

W0803 09:50:01.438162 139741922998144 deprecation_wrapper.py:119] From /usr/local/lib/python3.6/dist-packages/keras/optimizers.py:790: The name tf.train.Optimizer is deprecated. Please use tf.compat.v1.train.Optimizer instead.

W0803 09:50:01.468114 139741922998144 deprecation_wrapper.py:119] From /usr/local/lib/python3.6/dist-packages/keras/backend/tensorflow_backend.py:3295: The name tf.log is deprecated. Please use tf.math.log instead.

W0803 09:50:01.585267 139741922998144 deprecation.py:323] From /usr/local/lib/python3.6/dist-packages/tensorflow/python/ops/math_grad.py:1250: add_dispatch_support.<locals>.wrapper (from tensorflow.python.ops.array_ops) is deprecated and will be removed in a future version.
Instructions for updating:
Use tf.where in 2.0, which has the same broadcast rule as np.where
W0803 09:50:02.847234 139741922998144 deprecation_wrapper.py:119] From /usr/local/lib/python3.6/dist-packages/keras/backend/tensorflow_backend.py:986: The name tf.assign_add is deprecated

Epoch 1/25
39978/39978 [==============================] - 31s 788us/step - loss: 0.7877
Epoch 2/25
39978/39978 [==============================] - 30s 746us/step - loss: 0.7467
Epoch 3/25
39978/39978 [==============================] - 30s 748us/step - loss: 0.7210
Epoch 4/25
39978/39978 [==============================] - 29s 735us/step - loss: 0.6946
Epoch 5/25
39978/39978 [==============================] - 30s 740us/step - loss: 0.6742
Epoch 6/25
39978/39978 [==============================] - 30s 739us/step - loss: 0.6540
Epoch 7/25
39978/39978 [==============================] - 30s 743us/step - loss: 0.6363
Epoch 8/25
39978/39978 [==============================] - 30s 744us/step - loss: 0.6208
Epoch 9/25
39978/39978 [==============================] - 30s 747us/step - loss: 0.6080
Epoch 10/25
39978/39978 [==============================] - 30s 750us/step - loss: 0.5942
Epoch 11/25
39978/39978 [==============================] - 30s 746us/step - loss: 0.5816
Epoch 12/25
39978/39978 [=====

In [0]:
y_pred = model.predict([x_val, x_heading_val])

In [55]:
score = model.evaluate([x_val, x_heading_val],y_val)
print(model.metrics_names, ":", score)

9994/9994 [==============================] - 5s 471us/step
['loss'] : 0.5833491314547716


## Build the same model with attention layers included for better performance (Optional)

## Fit the model and report the accuracy score for the model with attention layer (Optional)